In [1]:
from tokenize import cookie_re

from metpy.testing import assert_xarray_allclose
from osgeo import gdal
import rioxarray
import numpy as np
import pyhdf
from pyhdf.SD import SD, SDC
import xarray as xr
import matplotlib.pyplot as plt
import satpy
from satpy import Scene, find_files_and_readers
from satpy.enhancements.enhancer import get_enhanced_image
from pathlib import Path
from glob import glob
from datetime import datetime

In [2]:
def plot_difference_histogram(
    corrected,
    uncorrected,
    *,
    percent=False,
    bins=100,
    range_percentile=99,
    title=None,
    ax=None
):
    """
    Plot a histogram of band differences (corrected - uncorrected).

    Parameters
    ----------
    corrected : ndarray
        Corrected band data
    uncorrected : ndarray
        Uncorrected band data
    percent : bool, optional
        If True, plot percent difference: 100 * (c - u) / u
    bins : int, optional
        Number of histogram bins
    range_percentile : float, optional
        Percentile used to limit histogram range (symmetric)
    title : str, optional
        Plot title
    ax : matplotlib axis, optional
        Existing axis to plot on

    Returns
    -------
    stats : dict
        Dictionary of summary statistics
    """

    # Compute difference
    if percent:
        with np.errstate(divide="ignore", invalid="ignore"):
            diff = 100.0 * (corrected - uncorrected) / uncorrected
        xlabel = "Percent difference (%)"
    else:
        diff = np.array(corrected - uncorrected)
        xlabel = "Difference (corrected − uncorrected)"

    # Flatten and remove NaNs/Infs
    diff = diff.ravel()
    diff = diff[np.isfinite(diff)]

    # Robust range using percentiles
    p = np.nanpercentile(np.abs(diff), range_percentile)
    hist_range = (-p, p)

    # Create axis if needed
    if ax is None:
        fig, ax = plt.subplots(figsize=(7, 4))

    # Plot histogram
    ax.hist(diff, bins=bins, range=hist_range, histtype="stepfilled", alpha=0.7)

    # Statistics
    mean = np.nanmean(diff)
    median = np.nanmedian(diff)
    std = np.nanstd(diff)
    rmse = np.sqrt(np.nanmean(diff**2))

    # Overlay statistics
    ax.axvline(mean, linestyle="--", linewidth=2, label=f"Mean = {mean:.3g}")
    ax.axvline(median, linestyle=":", linewidth=2, label=f"Median = {median:.3g}")

    # Labels
    ax.set_xlabel(xlabel)
    ax.set_ylabel("Pixel count")
    ax.legend()

    if title is not None:
        ax.set_title(title)

    stats = {
        "mean": mean,
        "median": median,
        "std": std,
        "rmse": rmse,
        "n_pixels": diff.size,
    }

    return stats


In [3]:
import warnings
warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    module="dask"
)


In [4]:
import os
from pathlib import Path

# Create symlink to fix the naming issue
pyspectral_dir = Path.home() / 'Library/Application Support/pyspectral'
source = pyspectral_dir / 'rsr_modis_EOS-Aqua.h5'
target = pyspectral_dir / 'rsr_modis_Aqua.h5'

if source.exists() and not target.exists():
    os.symlink(source, target)
    print(f"Created symlink: {target} -> {source}")
else:
    print(f"Source exists: {source.exists()}, Target exists: {target.exists()}")

Source exists: True, Target exists: True


In [5]:
DATA_DIR = str(Path('~/Downloads').expanduser())

####  --- MODIS ---

https://github.com/pytroll/satpy/blob/main/satpy/etc/composites/modis.yaml

In [5]:
myfiles = find_files_and_readers(base_dir=DATA_DIR,
                                 sensor="modis",
                                 start_time=datetime(2026, 1, 1, 12, 30),
                                 end_time=datetime(2026, 1, 1, 12, 30),
                                 reader='modis_l1b',)

scn = Scene(filenames=myfiles)

In [ ]:
d# scn.load(['solar_zenith_angle', 'satellite_zenith_angle',
#           'satellite_azimuth_angle', 'solar_azimuth_angle'])
scn.load(scn.available_dataset_names()[:-1])

In [ ]:
scn.load(['true_color_uncorrected'])
scn.load(['true_color'])

In [ ]:
# scn.load(['1','4', '3'], modifiers=('sunz_corrected', 'rayleigh_corrected'))

In [ ]:
scn_resampled = scn.resample(resampler='native')
print(scn_resampled)

In [ ]:
print(scn_resampled['1'].attrs['modifiers'])

In [ ]:
scn.load(['1', '3', '4'], modifiers=('sunz_corrected', 'rayleigh_corrected'))

In [ ]:
correction_applied_bands = scn.resample(resampler='native')

In [ ]:
correction_applied_bands['1'].attrs['modifiers']

In [ ]:
scn.unload(['1', '3', '4'])

In [ ]:
scn.load(['1', '3', '4'])

In [ ]:
uncorrected_bands = scn.resample(resampler='native')

In [ ]:
_band1_boolean = correction_applied_bands['1'] == uncorrected_bands['1']

In [ ]:
np_boolean_results = np.array(_band1_boolean)

In [ ]:
numpy_true_count = np.sum(np_boolean_results)
numpy_false_count = np.sum(~np_boolean_results)


In [ ]:
numpy_false_count

In [ ]:
numpy_true_count

In [ ]:
_band1_corr_minus_uncorr = correction_applied_bands['1'] - uncorrected_bands['1']

In [ ]:
plot_difference_histogram(corrected=correction_applied_bands['1'], uncorrected=uncorrected_bands['1'])

### --- VIIRS ---

https://github.com/pytroll/satpy/blob/main/satpy/etc/composites/viirs.yaml

In [ ]:
viirs_files = find_files_and_readers(base_dir=DATA_DIR,
                                     start_time=datetime(2026, 1, 1, 12, 00),
                                     end_time=datetime(2026, 1, 1, 12, 00),
                                     reader='viirs_l1b',)
print(viirs_files)
viirs_scn = Scene(filenames=viirs_files)
print(viirs_scn.all_dataset_names())

In [ ]:
viirs_data_load_list = viirs_scn.all_dataset_names()[1:22] + viirs_scn.all_dataset_names()[-8:]
viirs_scn.load(viirs_data_load_list)

In [ ]:
resampled_viirs = viirs_scn.resample(resampler='native')

In [ ]:
resampled_viirs['M01'].attrs['modifiers']

In [ ]:
viirs_scn.load(['M01'], modifiers=('sunz_corrected', 'rayleigh_corrected'))

In [ ]:
m01_corrected = viirs_scn.resample(resampler='native')

In [ ]:
m01_corrected['M01'].attrs['modifiers']

### --- SENTINEL-2 ---

https://github.com/pytroll/satpy/blob/main/satpy/etc/composites/sen2_msi.yaml

In [7]:
DATA_DIR

'/Users/cwelch/Downloads'

In [8]:
file_dir = DATA_DIR + '/S2A_MSIL1C_20260101T125331_N0511_R052_T23KRR_20260101T142917.SAFE'

In [6]:
msi_files = find_files_and_readers(base_dir=DATA_DIR,
                                     start_time=datetime(2026, 1, 1, 12, 00),
                                     end_time=datetime(2026, 1, 1, 12, 00),
                                     reader='msi_safe',)
print(msi_files)
msi_scn = Scene(filenames=msi_files)
print(msi_scn.all_dataset_names())

{'msi_safe': ['/Users/cwelch/Downloads/S2A_MSIL1C_20260101T125331_N0511_R052_T23KRR_20260101T142917.SAFE/GRANULE/L1C_T23KRR_A054990_20260101T125328/MTD_TL.xml', '/Users/cwelch/Downloads/S2A_MSIL1C_20260101T125331_N0511_R052_T23KRR_20260101T142917.SAFE/MTD_MSIL1C.xml', '/Users/cwelch/Downloads/S2A_MSIL1C_20260101T125331_N0511_R052_T23KRR_20260101T142917.SAFE/GRANULE/L1C_T23KRR_A054990_20260101T125328/IMG_DATA/T23KRR_20260101T125331_B8A.jp2', '/Users/cwelch/Downloads/S2A_MSIL1C_20260101T125331_N0511_R052_T23KRR_20260101T142917.SAFE/GRANULE/L1C_T23KRR_A054990_20260101T125328/IMG_DATA/T23KRR_20260101T125331_B10.jp2', '/Users/cwelch/Downloads/S2A_MSIL1C_20260101T125331_N0511_R052_T23KRR_20260101T142917.SAFE/GRANULE/L1C_T23KRR_A054990_20260101T125328/IMG_DATA/T23KRR_20260101T125331_B04.jp2', '/Users/cwelch/Downloads/S2A_MSIL1C_20260101T125331_N0511_R052_T23KRR_20260101T142917.SAFE/GRANULE/L1C_T23KRR_A054990_20260101T125328/IMG_DATA/T23KRR_20260101T125331_B05.jp2', '/Users/cwelch/Downloads/S2

In [13]:
print(len(msi_scn.all_dataset_names()[:-4]))


13


In [15]:
print(msi_scn.available_composite_names())

['cloud_phase', 'cloud_phase_raw', 'dataspace_swir', 'dataspace_swir_uncorr', 'day_essl_colorized_low_level_moisture', 'day_essl_low_level_moisture', 'essl_colorized_low_level_moisture', 'essl_low_level_moisture', 'false_color', 'natural_color', 'ndmi', 'ndsi', 'ndsi_with_true_color', 'ndvi', 'ndwi', 'true_color', 'true_color_antarctic', 'true_color_continental_average', 'true_color_continental_clean', 'true_color_continental_polluted', 'true_color_desert', 'true_color_marine_clean', 'true_color_marine_polluted', 'true_color_marine_tropical', 'true_color_raw', 'true_color_rural', 'true_color_uncorr', 'true_color_urban', 'urban_color']
